In [33]:
#!pip install eli5==0.13.0
#!apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [34]:
'''
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance
'''

In [42]:
import pandas as pd

In [43]:
train_path = '/data/ephemeral/home/train.csv'
test_path  = '/data/ephemeral/home/test.csv'
subway_path = '/data/ephemeral/home/subway_feature.csv'
bus_path = '/data/ephemeral/home/bus_feature.csv'
sub_path = '/data/ephemeral/home/realprice_file.csv'
sub_data = pd.read_csv(sub_path)
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
subway = pd.read_csv(subway_path)
bus = pd.read_csv(bus_path)

In [44]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
df_train['is_test'] = 0
df_test['is_test'] = 1
concat = pd.concat([df_train, df_test])

In [45]:
concat['is_test'].value_counts()

0    1118822
1       9272
Name: is_test, dtype: int64

In [46]:

# '거래년월' 변수의 데이터 개수 확인
transaction_counts = concat['계약년월'].value_counts().reset_index()
transaction_counts.columns = ['계약년월', '데이터 개수']


In [47]:
# 'transaction_counts'를 새로운 데이터프레임으로 만들기
df_transaction_counts = pd.DataFrame(transaction_counts)


In [48]:
print("새로운 데이터프레임:")
print(df_transaction_counts.head(100))

새로운 데이터프레임:
      계약년월  데이터 개수
0   202006   16364
1   201503   15510
2   201808   14990
3   201707   14922
4   201705   14766
..     ...     ...
95  201708    5126
96  201306    5113
97  202105    5011
98  201601    4998
99  200706    4980

[100 rows x 2 columns]


In [49]:
df_transaction_counts

,계약년월,데이터 개수
0,202006,16364
1,201503,15510
2,201808,14990
3,201707,14922
4,201705,14766
...,...,...
196,202207,690
197,202211,646
198,202209,620
199,202210,576


In [50]:

# 'df_transaction_counts'와 'sub_data'를 '계약년월'을 기준으로 합치기
merged_df = pd.merge(df_transaction_counts, sub_data, on='계약년월', how='left')

print("합친 데이터프레임:")
print(merged_df)

합친 데이터프레임:
       계약년월  데이터 개수  실거래지수
0    202006   16364  140.2
1    201503   15510   79.4
2    201808   14990  115.0
3    201707   14922   97.7
4    201705   14766   93.4
..      ...     ...    ...
196  202207     690  173.7
197  202211     646  147.8
198  202209     620  164.4
199  202210     576  156.6
200  201609     460   90.4

[201 rows x 3 columns]


In [51]:
merged_df_no_missing = merged_df.dropna()

print("결측치를 제거한 데이터프레임:")
print(merged_df_no_missing)

결측치를 제거한 데이터프레임:
       계약년월  데이터 개수  실거래지수
0    202006   16364  140.2
1    201503   15510   79.4
2    201808   14990  115.0
3    201707   14922   97.7
4    201705   14766   93.4
..      ...     ...    ...
196  202207     690  173.7
197  202211     646  147.8
198  202209     620  164.4
199  202210     576  156.6
200  201609     460   90.4

[119 rows x 3 columns]


In [53]:
# 데이터 개수와 '실거래지수'를 곱한 결과를 'new' 변수로 나누기
merged_df_no_missing['new'] = (merged_df_no_missing['데이터 개수'] * merged_df_no_missing['실거래지수'])

print("새로운 열 'new'를 추가한 결과:")
print(merged_df_no_missing)

새로운 열 'new'를 추가한 결과:
       계약년월  데이터 개수  실거래지수        new
0    202006   16364  140.2  2294232.8
1    201503   15510   79.4  1231494.0
2    201808   14990  115.0  1723850.0
3    201707   14922   97.7  1457879.4
4    201705   14766   93.4  1379144.4
..      ...     ...    ...        ...
196  202207     690  173.7   119853.0
197  202211     646  147.8    95478.8
198  202209     620  164.4   101928.0
199  202210     576  156.6    90201.6
200  201609     460   90.4    41584.0

[119 rows x 4 columns]


In [54]:
# '계약년월'을 크기순으로 정렬
merged_df_no_missing_sorted = merged_df_no_missing.sort_values(by='계약년월')

print("계약년월을 크기순으로 정렬한 결과:")
print(merged_df_no_missing_sorted)

계약년월을 크기순으로 정렬한 결과:
       계약년월  데이터 개수  실거래지수       new
91   201311    5269   74.5  392540.5
66   201312    6504   74.3  483247.2
49   201401    7247   75.0  543525.0
36   201402    8798   75.6  665128.8
44   201403    8010   76.0  608760.0
..      ...     ...    ...       ...
148  202305    3500  153.5  537250.0
133  202306    3867  156.3  604412.1
144  202307    3695  157.9  583440.5
134  202308    3864  159.8  617467.2
180  202309    1713  161.4  276478.2

[119 rows x 4 columns]


In [75]:

# 202001~202306까지의 'new' 값을 더한 후 데이터 개수로 나누기
start_year_month = '202003'
end_year_month = '202306'

# '계약년월' 열을 문자열로 변환하여 비교
filtered_merged_df = merged_df_no_missing_sorted[
    (merged_df_no_missing_sorted['계약년월'].astype(str) >= start_year_month) & 
    (merged_df_no_missing_sorted['계약년월'].astype(str) <= end_year_month)
]

total_new = filtered_merged_df['new'].sum()
average_new = total_new / filtered_merged_df['데이터 개수'].sum()

print(f"{start_year_month}부터 {end_year_month}까지의 'new' 값의 평균: {average_new}")

202003부터 202306까지의 'new' 값의 평균: 158.9664615254321
